In [15]:
import jax.numpy as np
import jax.random as random
from jax import grad, jit, vmap

key = random.PRNGKey(42)

import matplotlib.pyplot as plt

In [7]:
import gym

In [3]:
print(random.normal(key, shape=(3,)))  # [ 1.81608593 -0.48262325  0.33988902]

[ 0.18693541 -1.2806509  -1.5593134 ]


In [4]:
# WANT: to show abstractions emerging from heirarchical MPC

In [ ]:
def mpc(initial_state, reward_fn, transition_fn, T):
    # solve it ...
    # is there some sort of solver I can use here?
    # differentiable MPC would be great!
    for t in T:
        
    return actions

In [29]:
class MPCLearner():
    """
    No memoization. Nothing fancy.
    Set subgoals, use MPC to achieve them.
    """
    def __init__(self, x):
        self.policy = None
        self.transition_fn = None
        self.metric = None
        
    def __call__(self, state):
        """
        Args:
            state: the current state
                
        Returns:
            a: the action to take
        """
        if self.metric(state, goal) > 0.9: 
            # if the goal has been reached
            # pick a new one
            self.goal = self.policy(state)
            # no need to reward the lower level learner here as it is just a dict.

        self.plan = self.mpc(state, self.transition_fn, self.plan, lambda s: self.metric(s, goal), T=10)
        
        a = self.plan.pop_left()
        # measure of surprise is a sharp change in the plan!?
        
        return a

In [ ]:
# now construct a heirarchy of these. each with different T and goal-action spaces!?

class MPCPlanner():
    def __init__(self, x):
        self.policy = None
        self.transition_fn = None
        self.metric = None
        
        self.action_space = None
        
    def __call__(self, state, goal):
        # needs a call back!? when goal has been approx reached!?
        self.plan = self.mpc(state, self.transition_fn, self.plan, lambda s: self.metric(s, goal), T=10)
        a = self.plan.pop_left()
        # measure of surprise is a sharp change in the plan!?
        return a
    
    
class Heirarchy():
    def __init__(self):
        self.layers = [MPCPlanner() for _ in range(3)]
        
    def __call__(self, state):
        if self.metric(state, goal) > 0.9: 
            # if the goal has been reached
            # pick a new one
            self.goal = self.policy(state)
            # no need to reward the lower level learner here as it is just a dict.
        
        sub_goal = self.goal
        for layer in self.layers:
            sub_goal = layer(state, sub_goal)
            
        return sub_goal

In [5]:
class MemoDict(dict):
    # TODO should only work on the first two args; states and goals.
    # not that doesnt make sense...
    
    # the key to making this useful will be using notions of similarity
    # so we can reuse old solutions for slightly different state-goal-transition-...s
    
    # what about compressing the memory?
    # using function approximation instead. this would mean ...!?
    
    # this is the sort of thing you would want to wrap every function with!?
    def __init__(self, f):
        self.f = f
    def __call__(self, *args):
        return self[args]
    def __missing__(self, key):
        # TODO want to add based on density / rate.
        # if the 'same thing' keeps being solved then just remember the answer
        ret = self[key] = self.f(*key)
        return ret

In [8]:
class LazyLearner():
    def __init__(self, x):
        # the mpc will end up storing a bunch of options / sub policies
        self.mpc = MemoDict(mpc)  
        
        self.policy = None
        self.transition_fn = None
        self.metric = None
        
    def __call__(self, state, goal_fn):
        """
        Args:
            state: the current state
            goal_fn: a callable fn that returns the expected return. goal_fn: s -> R.
                (could be the estimated value fn, or a subgoal / metric from a high lvl policy!?)
                (but how can it be used!?)
                
        Returns:
            a: the action to take
        """
        if self.metric(state, goal) > 0.9: 
            # if the goal has been reached
            # pick a new one
            self.goal = self.policy(state)
            # no need to reward the lower level learner here as it is just a dict.

        self.plan = self.mpc(state, self.transition_fn, self.plan, lambda s: self.metric(s, goal), T=10)
        
        a = self.plan.pop_left()
        # measure of surprise is a sharp change in the plan!?
        
        return a